## Shortest Path Problem: Susan Winslow

In [23]:
# import Glop package
from ortools.linear_solver import pywraplp as glp
import lptools as lpt

In [24]:
#Create LP model object
mymodel = glp.Solver('ShortestPath',glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [18]:
inf = mymodel.infinity()

# node dictionary; value = -1 is origin, value = 1 is destination, value = 0 is intermediate node
node = {1:-1,2:0,3:0,4:0,5:0,6:1 }

# edge dictionary; (origin,destination):distance/cost
edge = {(1,2):80,
        (1,5):130,
        (1,3):40,
        (1,6):180,
        (1,4):80,
        (2,5):60,
        (5,2):60,
        (3,5):90,
        (5,3):90,
        (3,4):30,
        (4,3):30,
        (4,5):50,
        (5,4):50,
        (3,6):120,
        (2,6):100,
        (5,6):30,
        (4,6):90}

mymodel.Objective().SetMinimization()    # minimize total transportation cost

In [19]:
# create a constraint dictionary containing a constraint for each node
constr = dict()
for n in  node:
    b =  node[n]
    constr[n] = mymodel.Constraint(b,b,(str(n)) )

In [20]:
# create a variable dictionary containing a variable for each edge
# add each variable to the objective and its corresponding constraints

path = dict()
for e in edge:
    (o,d) = e
    c = edge[e]
    path[e] = mymodel.NumVar(0,inf, str(o) + '_' + str(d))
    mymodel.Objective().SetCoefficient(path[e],c)
    constr[o].SetCoefficient(path[e],-1)
    constr[d].SetCoefficient(path[e],1)

In [21]:
lpt.print_model(mymodel)

Variables:
1_2, 1_5, 1_3, 1_6, 1_4, 2_5, 5_2, 3_5, 5_3, 3_4, 4_3, 4_5, 5_4, 3_6, 2_6, 5_6, 4_6 

minimize: 80.0*1_2 + 130.0*1_5 + 40.0*1_3 + 180.0*1_6 + 80.0*1_4 + 60.0*2_5 + 60.0*5_2 + 90.0*3_5 + 90.0*5_3 + 30.0*3_4 + 30.0*4_3 + 50.0*4_5 + 50.0*5_4 + 120.0*3_6 + 100.0*2_6 + 30.0*5_6 + 90.0*4_6 

Subject To:
1: - 1.0*1_2 - 1.0*1_5 - 1.0*1_3 - 1.0*1_6 - 1.0*1_4 = -1.0
2: 1.0*1_2 - 1.0*2_5 + 1.0*5_2 - 1.0*2_6 = 0.0
3: 1.0*1_3 - 1.0*3_5 + 1.0*5_3 - 1.0*3_4 + 1.0*4_3 - 1.0*3_6 = 0.0
4: 1.0*1_4 + 1.0*3_4 - 1.0*4_3 - 1.0*4_5 + 1.0*5_4 - 1.0*4_6 = 0.0
5: 1.0*1_5 + 1.0*2_5 - 1.0*5_2 + 1.0*3_5 - 1.0*5_3 + 1.0*4_5 - 1.0*5_4 - 1.0*5_6 = 0.0
6: 1.0*1_6 + 1.0*3_6 + 1.0*2_6 + 1.0*5_6 + 1.0*4_6 = 1.0

Bounds:
1_2 >= 0.0
1_5 >= 0.0
1_3 >= 0.0
1_6 >= 0.0
1_4 >= 0.0
2_5 >= 0.0
5_2 >= 0.0
3_5 >= 0.0
5_3 >= 0.0
3_4 >= 0.0
4_3 >= 0.0
4_5 >= 0.0
5_4 >= 0.0
3_6 >= 0.0
2_6 >= 0.0
5_6 >= 0.0
4_6 >= 0.0


In [22]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f' % mymodel.Objective().Value())
for v in mymodel.variables():
    if v.solution_value() != 0:
        print('%s = %.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 150.00
1_3 = 1.00
3_4 = 1.00
4_5 = 1.00
5_6 = 1.00


In [ ]:
# display all variable information
print('Variable    LB   Value    UB   Reduced Cost')
for v in mymodel.variables():
    print('%8s  %5.1f  %5.1f  %5.1f  %5.2f' % (v.name(),v.lb(),v.solution_value(),v.ub(),v.reduced_cost()))

In [ ]:
#display constraint information
print('Constraint    LB    Value  UB     Dual')
for (c,lhs) in zip(mymodel.constraints(),mymodel.ComputeConstraintActivities()):
    print('%10s  %5.1f  %5.1f  %5.1f  %5.2f' % (c.name(),c.lb(),lhs,c.ub(),c.dual_value()))